## Tuning Neural Networks

In [1]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import RMSprop
from torch.utils.data import random_split, DataLoader, TensorDataset
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Create training and test sets
features, target = make_classification(n_classes=2, n_features=10,
n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
features, target, test_size=0.1, random_state=1)

# Set random seed
torch.manual_seed(0)
np.random.seed(0)

# Convert data to PyTorch tensors
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)
# Define a neural network using `Sequential`

class SimpleNeuralNet(nn.Module):
    def __init__(self, layer_size_1=10, layer_size_2=10):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
        torch.nn.Linear(10, layer_size_1),
        torch.nn.ReLU(),
        torch.nn.Linear(layer_size_1, layer_size_2),
        torch.nn.ReLU(),
        torch.nn.Linear(layer_size_2, 1),
        torch.nn.Sigmoid()
        )
    
    def forward(self, x):
            x = self.sequential(x)
            return x


config = {
    "layer_size_1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "layer_size_2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
}

scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=1000,
        grace_period=1,
        reduction_factor=2
)
reporter = CLIReporter(
    parameter_columns=["layer_size_1", "layer_size_2", "lr"],
    metric_columns=["loss"]
)

ImportError: cannot import name 'MapTransformFn' from 'ray.data._internal.execution.interfaces' (/home/isabella/miniconda3/envs/ml-zoomcamp/lib/python3.9/site-packages/ray/data/_internal/execution/interfaces/__init__.py)

In [2]:
# # Train neural network
def train_model(config, epochs=3):
    network = SimpleNeuralNet(config["layer_size_1"], config["layer_size_2"])
    criterion = nn.BCELoss()
    optimizer = optim.SGD(network.parameters(), lr=config["lr"], momentum=0.9)
    train_data = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_data, batch_size=100, shuffle=True)
    # Compile the model using torch 2.0's optimizer
    network = torch.compile(network)

    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = network(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            tune.report(loss=(loss.item()))

In [3]:
result = tune.run(
    train_model,
    resources_per_trial={"cpu": 2},
    config=config,
    num_samples=1,
    scheduler=scheduler,
    progress_reporter=reporter
)

2023-12-28 00:13:26,177	ERROR services.py:1207 -- Failed to start the dashboard , return code 0
2023-12-28 00:13:26,181	ERROR services.py:1232 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/ray-logging.html#logging-directory-structure' to find where the log file is.
2023-12-28 00:13:26,183	ERROR services.py:1276 -- 
The last 20 lines of /tmp/ray/session_2023-12-28_00-13-23_651130_140813/logs/dashboard.log (it contains the error message from the dashboard): 
    return future.result()
  File "/home/isabella/miniconda3/envs/ml-zoomcamp/lib/python3.9/site-packages/ray/dashboard/dashboard.py", line 75, in run
    await self.dashboard_head.run()
  File "/home/isabella/miniconda3/envs/ml-zoomcamp/lib/python3.9/site-packages/ray/dashboard/head.py", line 327, in run
    self.http_server = await self._configure_http_server(modules)
  File "/home/isabella/miniconda3/envs/ml-zoomcamp

: 

: 

In [ ]:
best_trial = result.get_best_trial("loss", "min", "last")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
best_trial.last_result["loss"]))
best_trained_model = SimpleNeuralNet(best_trial.config["layer_size_1"],
best_trial.config["layer_size_2"])